In [ ]:
'''
Author: Andrew H. Fagg
'''
%reload_ext autoreload
%autoreload 2

import sys
tf_tools = "../../../../../tf_tools/"
#sys.path.append(tf_tools + "metrics")
#sys.path.append(tf_tools + "networks")
sys.path.append(tf_tools + "experiment_control")

#from job_control import *
import argparse
import pickle
import random
from chesapeake_data import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Concatenate, UpSampling2D, Add
from tensorflow.keras.layers import Convolution2D, Dense, MaxPooling2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras import Input, Model

#################################################################
# Default plotting parameters
FIGURESIZE=(10,6)
FONTSIZE=18

plt.rcParams['figure.figsize'] = FIGURESIZE
plt.rcParams['font.size'] = FONTSIZE

plt.rcParams['xtick.labelsize'] = FONTSIZE
plt.rcParams['ytick.labelsize'] = FONTSIZE

#################################################################


In [ ]:
def create_network(input_shape, nclasses, filters=[30,30],
                   lambda_regularization=None, activation='elu'):
    if lambda_regularization is not None:
        lambda_regularization = keras.regularizers.l2(lambda_regularization)
        
    input_tensor = Input(shape=input_shape, name="input")
        
    tensor = Convolution2D(filters[0],
                          kernel_size=(3,3),
                           padding='same', 
                          use_bias=True,
                          kernel_initializer='random_uniform',
                          bias_initializer='zeros',
                          kernel_regularizer=lambda_regularization,
                          activation=activation)(input_tensor)
    
    
    # Fill in magic
    

    model = Model(inputs=input_tensor, outputs=output_tensor)
    opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, 
                                    epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=opt,
                 metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    
    return model


def training_set_generator_images(ins, outs, batch_size=10,
                          input_name='input', 
                        output_name='output'):
    '''
    Generator for producing random minibatches of image training samples.
    
    @param ins Full set of training set inputs (examples x row x col x chan)
    @param outs Corresponding set of sample (examples x nclasses)
    @param batch_size Number of samples for each minibatch
    @param input_name Name of the model layer that is used for the input of the model
    @param output_name Name of the model layer that is used for the output of the model
    '''
    
    while True:
        # Randomly select a set of example indices
        example_indices = random.choices(range(ins.shape[0]), k=batch_size)
        
        # The generator will produce a pair of return values: one for inputs and one for outputs
        yield({input_name: ins[example_indices,:,:,:]},
             {output_name: outs[example_indices,:,:]})

  

In [ ]:
file_base = '/home/fagg/datasets/radiant_earth/pa/pa_1m_2013_extended-train_patches/'
ins, mask, outs, weights = load_files_from_dir(file_base + "/F0", filt='-[01]?')
class_hist, class_weights=compute_class_weights(outs)

## Model

In [ ]:
model = create_network(ins.shape[1:4], nclasses=7)
print(model.summary())

In [ ]:
generator = training_set_generator_images(ins, outs, batch_size=50)

In [ ]:
history = model.fit(x=generator, epochs=10, steps_per_epoch=2,
                        use_multiprocessing=False, 
                        verbose=True)
